In [5]:

####################################################################################################
####################################################################################################

import sys
import os.path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

import keras
import keras.utils.vis_utils
from keras import backend as K
from keras import layers
from keras.layers import Dense
from keras.models import load_model
from keras.models import Sequential
from keras.utils import plot_model

from helpers import *

sys.path.append(os.path.abspath("../"))
from pax_utils import s1s2_utils



In [6]:

####################################################################################################
####################################################################################################

#file_all       = "../pax_merge/merged/merged_all_1803031449.pkl"
file_all = "../data_lfs/merged_all_50k_1803141941.pkl"

df_all = pd.read_pickle(file_all)
df_all = df_all[df_all['intr_count'] == 1].reset_index(drop=True)
df_all = df_all[25000:][:]

df_train_truth = df_all[:][['x', 'y']]
df_train_input = df_all[:][s1s2_utils.getS2integralsDataFrameColumns()]


####################################################################################################
####################################################################################################

checkTrainingData(df_train_input, df_train_truth)




Input shape: (5011, 127)
Truth shape: (5011, 2)

Input DataFrame: 


,s2_area_000,s2_area_001,s2_area_002,s2_area_003,s2_area_004,s2_area_005,s2_area_006,s2_area_007,s2_area_008,s2_area_009,...,s2_area_117,s2_area_118,s2_area_119,s2_area_120,s2_area_121,s2_area_122,s2_area_123,s2_area_124,s2_area_125,s2_area_126
25000,0.825528,0.0,0.0,1.152132,0.000000,0.000000,0.000000,0.000000,0.972961,0.000000,...,4.937289,0.0,7.218913,1.425349,0.183338,2.849777,0.465242,7.050439,3.856088,1.239840
25001,4.260632,0.0,0.0,9.810595,13.234727,15.455276,20.949707,19.900917,16.701942,3.698376,...,2.916543,0.0,5.677208,16.205469,14.773957,6.865777,8.004706,4.175336,6.680394,1.746098
25002,1.804438,0.0,0.0,7.816449,21.015508,7.980388,6.982829,2.663670,1.203736,1.976867,...,1.906674,0.0,3.990904,3.151415,7.477839,3.045131,1.434088,2.635961,1.595465,1.373961
25003,2.391124,0.0,0.0,1.232717,1.391422,0.000000,0.275137,0.000000,0.996703,0.811028,...,0.000000,0.0,34.773565,1.633377,1.328085,0.000000,0.000000,0.815733,5.764327,3.016163
25004,0.798541,0.0,0.0,0.412054,0.774462,5.975792,3.433039,0.000000,0.827500,2.705991,...,1.307462,0.0,25.923236,65.688856,11.813069,5.744365,4.154686,1.235295,15.566745,9.266530



Truth DataFrame:


,x,y
25000,-14.652233,28.887292
25001,19.758080,19.924424
25002,16.882350,30.988062
25003,-19.448916,12.724450
25004,-1.879574,17.775761


In [9]:

####################################################################################################
####################################################################################################

folder         = "models" + "/"    
model_name     = "model_barge_xy_elu"
#model_name     = "model_xenon1t_elu"
model_name_h5  = folder + model_name + ".h5"
model_name_png = folder + model_name + ".png"



In [11]:

####################################################################################################
####################################################################################################

model       = load_model(model_name_h5)
arr_xy_pred = model.predict(df_train_input.values)

print("\nLoaded Model: " + model_name_h5)
print("Predicted\n")




Loaded Model: models/model_barge_xy_elu.h5
Predicted



In [12]:

####################################################################################################
####################################################################################################

arr_x_true = df_train_truth[:]['x'].as_matrix()
arr_y_true = df_train_truth[:]['y'].as_matrix()

arr_x_pred = arr_xy_pred[:, 0]
arr_y_pred = arr_xy_pred[:, 1]

arr_dx     = arr_x_true - arr_x_pred
arr_dy     = arr_y_true - arr_y_pred



In [14]:

####################################################################################################
####################################################################################################

file_out         = "./predictions/xy_S2integrals.pkl"

cols_out         = ['event_number', 'x', 'y', 'intr_x', 'intr_y', 'intr_x_nn', 'intr_y_nn']

df_out           = df_all[:][cols_out]
df_out['x_pred'] = arr_x_pred
df_out['y_pred'] = arr_y_pred


df_out.to_pickle(file_out)


####################################################################################################
####################################################################################################

df_test = pd.read_pickle(file_out)

display(df_test[0:5][:])
print()



,event_number,x,y,intr_x,intr_y,intr_x_nn,intr_y_nn,x_pred,y_pred
25000,41622,-14.652233,28.887292,-13.341479,27.555764,-13.307838,26.983139,-14.682321,28.794851
25001,41623,19.758080,19.924424,20.074561,18.578321,20.242099,18.882111,20.715342,19.301130
25002,41624,16.882350,30.988062,15.336466,27.306391,15.820212,27.598846,16.544344,28.335680
25003,41625,-19.448916,12.724450,-19.575815,12.343985,-19.570337,13.116863,-22.067802,16.052364
25004,41626,-1.879574,17.775761,-1.371554,15.585840,-1.334894,16.159689,-2.260344,16.576002
